# Wage analysis

Считаем файл `wage.csv` в DataFrame с названием `wage`

`person_id` - идентификатор человека

`gender` - пол, 0 - женский, 1 - мужской

`wage` - заработная плата в рублях

In [2]:
import pandas as pd

wage = pd.read_csv('wage.csv')

Поменяем колонку `gender` так, чтобы были записи `F` (female) и `M` (male) вместо 0 и 1

In [3]:
wage['gender'] = wage['gender'].map({0: 'F', 1: 'M'})

print(wage.head())

   person_id gender          wage
0          0      M  46793.603811
1          1      M  33481.575720
2          2      M  44523.699084
3          3      M  15995.576829
4          4      F  10282.631224


Посчитаем среднюю зарплату у мужчин и женщин

In [4]:
mean_wage_by_gender = wage.groupby('gender')['wage'].mean()

print(mean_wage_by_gender)

gender
F    40855.747261
M    46815.944005
Name: wage, dtype: float64


Теперь взглянем внимательнее на данные и обнаружим, что некоторые люди попали в выборку несколько раз. 
    1. Находим таких людей при помощи `value_counts()`
    2. Убедимся, что записи по ним с одинаковым `wage`. Отображаем количество разных значений при помощи `nunique()`
    3. Избавляемся от повторяющихся значений `drop_duplicates()`

In [5]:
# Найдем людей с несколькими записями
duplicates = wage['person_id'].value_counts()
print(duplicates[duplicates > 1])
# Проверим, что у дубликатов одинаковая зарплата
print(wage.loc[wage['person_id'].isin([1, 2])]['wage'].nunique())
# Удалим дубликаты
wage_deduped = wage.drop_duplicates(['person_id'])
print(wage_deduped)

person_id
17    2
16    2
15    2
14    2
13    2
12    2
Name: count, dtype: int64
2
      person_id gender           wage
0             0      M   46793.603811
1             1      M   33481.575720
2             2      M   44523.699084
3             3      M   15995.576829
4             4      F   10282.631224
...         ...    ...            ...
1001        995      M   66503.737185
1002        996      M    9972.956272
1003        997      F  104504.616392
1004        998      M   98927.903076
1005        999      M  108107.141368

[1000 rows x 3 columns]


Теперь посмотрим внимательнее на зарплаты и избавимся от бессмысленных значений

In [6]:
print(wage['wage'].describe())
wage = wage[wage['wage'] > 0]

count      1006.000000
mean      43865.468877
std       55683.570648
min     -287418.645743
25%       14502.933964
50%       27309.529498
75%       52035.342033
max      755320.874132
Name: wage, dtype: float64


Посмотрим на зарплату с учетом бонуса

In [7]:
bonus = pd.read_csv('bonus.csv', sep=';')
bonus.head()

,person_id,bonus
0,905,85059.638382
1,836,7703.346074
2,287,3120.269742
3,548,5347.987142
4,575,137257.490614


Чтобы посчитать итоговую зарплату, нам нужно по каждому человеку знать и оклад, и премию. Для этого надо будет сджойнить таблицы по `person_id`

In [8]:
df = pd.merge(wage, bonus, on='person_id', how='outer')

Наконец, давайте посчитаем итоговую зарплату
   1. Заменим отсутствующие записи в колонке `bonus` нулями
   2. Уберём людей без `wage` - это те "плохие" записи, от которых мы избавлялись на предыдущих шагах
   3. Сделаем новую колонку `total`, которая будет равна 12 окладам и премии
   4. Посчитаем среднюю и медианную итоговую зарплату в разрезе по полу

In [9]:
df['bonus'] = df['bonus'].fillna(0)
df = df[df['wage'] > 0]
df['total'] = (df['wage'] * 12) + df['bonus']
gender_wage_stats = df.groupby('gender')[['total']].agg(['mean', 'median'])
print(gender_wage_stats)

                total               
                 mean         median
gender                              
F       571147.044444  348904.084527
M       659426.140833  437499.824868


Сохраним `df` в файл `final_wages.csv`

In [10]:
df.to_csv('`nunique()`', index=False)